In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import chi2
import os
import sys
import argparse
from utils_baofit import WThetaModel, BAOFit, PathBAOFit

# Arguments
class Args:
    def __init__(self):
        self.include_wiggles = "y"
        self.dataset = "DESY6"
        self.weight_type = 1
        self.nz_flag = "fid"
        self.cov_type = "cosmolike_data"
        self.cosmology_template = "planck"
        self.cosmology_covariance = "planck"
        self.delta_theta = 0.2
        self.theta_min = 0.5
        self.theta_max = 5
        self.n_broadband = 3
        self.bins_removed = 'None'
        self.diag_only = "n"
        self.remove_crosscov = "n"
args = Args()
args.include_wiggles = '' if args.include_wiggles == 'y' else '_nowiggles'

# Galaxy bias dictionary
galaxy_bias = {0: 1.7984801248816065, 1: 1.8023169145559157, 2: 1.8102780823227076, 3: 1.9542868433512404, 4: 2.1103760953498414, 5: 2.4099120180419513}
galaxy_bias = {bin_z: 1 for bin_z in range(10)}

# Path for saving the BAO fits
path_baofit_instance = PathBAOFit(
    include_wiggles=args.include_wiggles,
    dataset=args.dataset,
    weight_type=args.weight_type,
    nz_flag=args.nz_flag,
    cov_type=args.cov_type,
    cosmology_template=args.cosmology_template,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    n_broadband=args.n_broadband,
    bins_removed=args.bins_removed,
    verbose=True
)
args.bins_removed = path_baofit_instance.bins_removed # it has been post-processed to create the path
nbins = path_baofit_instance.nbins # it also has the information about the number of tomographic bins

# 2. Load data w(theta) and covariance
from utils_data import WThetaCovarianceData
wtheta_covariance_data = WThetaCovarianceData(
    dataset=args.dataset,
    weight_type=args.weight_type,
    cov_type=args.cov_type,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    bins_removed=args.bins_removed,
    diag_only=args.diag_only,
    remove_crosscov=args.remove_crosscov,
    nbins=nbins
)

theta_data, wtheta_data, cov = wtheta_covariance_data.process()

# 3. Get the template wtheta
wtheta_model = WThetaModel(
    args.include_wiggles,
    args.nz_flag,
    args.cosmology_template,
    args.n_broadband,
    galaxy_bias, 
)

# 4. First fit attempt. We use curve_fit
wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
n_params = len(wtheta_model.names_params)
n_params_true = len(wtheta_model.names_params) - (1 + args.n_broadband) * len(args.bins_removed)
wtheta_template = wtheta_model.get_wtheta_template()

popt, pcov = curve_fit(wtheta_template, theta_data, wtheta_data_concatenated, p0=wtheta_model.p0, bounds=wtheta_model.bounds, sigma=cov, absolute_sigma=False)
err_params = np.sqrt(np.diag(pcov))

# Let's compute the chi2 and dof
wtheta_fit_concatenated = wtheta_template(theta_data, *popt)
diff = wtheta_data_concatenated - wtheta_fit_concatenated
chi_square = diff @ np.linalg.inv(cov) @ diff
dof = len(wtheta_data_concatenated) - n_params_true
for bin_z in args.bins_removed:
    dof -= len(indices_theta_allbins[bin_z])

print(f'Best-fit alpha = {popt[0]:.4f} ± {np.sqrt(pcov[0, 0]):.4f}')
print(f'chi2/dof = {chi_square:.4f}/{dof}')

# 5. With our pipeline
baofit = BAOFit(
    wtheta_model,
    theta_data,
    wtheta_data,
    cov,
    path_baofit_instance,
)
alpha_best, err_alpha, chi2_best, dof = baofit.fit()


All theta values are the same. Using the first one as an array.
Using fid n(z), which has 6 redshift bins
Best-fit alpha = 0.9633 ± 0.0291
chi2/dof = 83.8232/107
Saving output to: fit_results/DESY6_1/nzfid_covcosmolike_data_plancktemp_planckcov_deltatheta0.2_thetamin0.5_thetamax5_3broadband_binsremoved[]
Best-fit alpha = 0.9566 ± 0.0303
chi2/dof = 86.0162/107


In [ ]:
args.bins_removed

In [ ]:
wtheta_model.names_params

In [ ]:
args.bins_removed

In [ ]:
bins_removed

In [ ]:
0.0220/ 0.9528*100

In [ ]:
0.0249/ 0.9675*100

In [ ]:
wtheta_model.names_params

In [ ]:
path_template

In [ ]:
Best-fit alpha = ± 
chi2/dof = 91.8390/107